In [1]:
dbpath = '/mnt/c/Documents/outlook/'
dbfile = f'{dbpath}/emails3.db'

In [2]:
from zipfile import ZipFile
import logging
import sys
import argparse
from sqlalchemy import create_engine
import sqlalchemy.orm as orm

In [3]:
import attr
from lib import Email, Base

In [4]:
engine = create_engine(f'sqlite:///{dbfile}', echo=False)
Base.metadata.create_all(engine)

In [5]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [6]:
session = Session()
emails = session.query(Email).count()
emails

32639

In [7]:
first =session.query(Email).filter(Email.id == 1)
first

In [8]:
item = first.one()

In [9]:
# Esto es necesario para que los mensajes se guarden
# en UTF-8, y no en base64.
from email import charset
charset.add_charset('utf-8', charset.SHORTEST)

In [10]:
import mailbox
mdir = mailbox.Maildir(f'{dbpath}/mail', None, True)

#for item in session.query(Email.label).distinct():
#    mdir.add_folder(item.label.replace('/','.'))


In [11]:
maxid = -1
boxes = dict()
total = 1
accum = 0
msg = None
try:
    while total > 0:
        total = 0
        mails = session.query(Email).filter(Email.id>maxid).order_by(Email.id).limit(100)
        for msg in mails:
            folder = msg.label.replace("/", ".")
            box = boxes.get(folder, None)
            if box is None:
                box = mdir.add_folder(folder)
                boxes[folder] = box
            # This fails in Python 3.6 because of
            # https://bugs.python.org/issue35985,
            # https://bugs.python.org/issue33524
            box.add(msg.message())
            maxid = msg.id
            total += 1
        accum += total
        print("Saved %d messages" % accum)
except:
    raise

TypeError: slice indices must be integers or None or have an __index__ method